In [2]:
import pandas as pd
# load the bank transaction dataset
df = pd.read_csv('D1.csv')
# info and the first 10 transactions
print(df.info())
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1544 entries, 0 to 1543
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   patient_id  1544 non-null   object 
 1   global_num  975 non-null    float64
 2   date        1544 non-null   object 
 3   location    1544 non-null   object 
 4   latitude    1544 non-null   float64
 5   longitude   1544 non-null   float64
dtypes: float64(3), object(3)
memory usage: 72.5+ KB
None
    patient_id  global_num        date               location   latitude  \
0  P1000000501         2.0  22/04/2020  Chittorgarh_Rajasthan  24.879999   
1  P1000000501         2.0  24/04/2020  Ratnagiri_Maharashtra  16.994444   
2  P1000000502         5.0  26/04/2020     Pindwara_Rajasthan  24.794500   
3  P1000000502         5.0  27/04/2020    Raipur_Chhattisgarh  21.250000   
4  P1000000502         5.0  28/04/2020        Gokak_Karnataka  16.166700   
5  P1000000504         7.0  30/04/2020  Lucknow_Utta

In [14]:
df.loc[df['patient_id'] == 'P1100000501']


,patient_id,global_num,date,location,latitude,longitude
751,P1100000501,NaN,20/05/2020,Chirala_Andhra Pradesh,15.812074,80.355377
752,P1100000501,NaN,20/05/2020,Markapur_Andhra Pradesh,15.764501,79.259491
753,P1100000501,NaN,21/05/2020,Chalakudy_Kerala,10.311879,76.331978


In [8]:
# group by account, then list all services
transactions = df.groupby(['patient_id'])['location'].apply(list)
print(transactions.head(5))

patient_id
P1000000501       [Chittorgarh_Rajasthan, Ratnagiri_Maharashtra]
P1000000502    [Pindwara_Rajasthan, Raipur_Chhattisgarh, Goka...
P1000000504                              [Lucknow_Uttar Pradesh]
P1000000505                              [Lucknow_Uttar Pradesh]
P1000000506                                        [Delhi_Delhi]
Name: location, dtype: object


In [9]:
from apyori import apriori
# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.05))
# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({'Chalakudy_Kerala'}), support=0.05598243688254665, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Chalakudy_Kerala'}), confidence=0.05598243688254665, lift=1.0)]), RelationRecord(items=frozenset({'Channapatna_Karnataka'}), support=0.05598243688254665, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Channapatna_Karnataka'}), confidence=0.05598243688254665, lift=1.0)]), RelationRecord(items=frozenset({'Gokak_Karnataka'}), support=0.054884742041712405, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Gokak_Karnataka'}), confidence=0.054884742041712405, lift=1.0)]), RelationRecord(items=frozenset({'Kollam_Kerala'}), support=0.059275521405049394, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Kollam_Kerala'}), confidence=0.059275521405049394, lift=1.0)]), RelationRecord(items=frozenset({'Lucknow_Uttar Pradesh'}), suppor

In [10]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append(['patient_id'.join(rule.items_base), 'location'.join(rule.items_add),
                          rule_set.support, rule.confidence, rule.lift])
            
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support',
                                        'Confidence', 'Lift'])
        
result_df = convert_apriori_results_to_pandas_df(results)
print(result_df.head(20))

  Left_side              Right_side   Support  Confidence  Lift
0                  Chalakudy_Kerala  0.055982    0.055982   1.0
1             Channapatna_Karnataka  0.055982    0.055982   1.0
2                   Gokak_Karnataka  0.054885    0.054885   1.0
3                     Kollam_Kerala  0.059276    0.059276   1.0
4             Lucknow_Uttar Pradesh  0.086718    0.086718   1.0
5              Ranebennur_Karnataka  0.086718    0.086718   1.0
6             Ratnagiri_Maharashtra  0.062569    0.062569   1.0
7            Sardarshahar_Rajasthan  0.147091    0.147091   1.0


In [6]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(10))

  Left_side              Right_side   Support  Confidence  Lift
0                  Chalakudy_Kerala  0.055982    0.055982   1.0
1             Channapatna_Karnataka  0.055982    0.055982   1.0
2                   Gokak_Karnataka  0.054885    0.054885   1.0
3                     Kollam_Kerala  0.059276    0.059276   1.0
4             Lucknow_Uttar Pradesh  0.086718    0.086718   1.0
5              Ranebennur_Karnataka  0.086718    0.086718   1.0
6             Ratnagiri_Maharashtra  0.062569    0.062569   1.0
7            Sardarshahar_Rajasthan  0.147091    0.147091   1.0


In [7]:
transactions = df.groupby(['patient_id'])['location'].apply(list)
sequences = transactions.values.tolist()
# show the first 5 sequences
print(sequences[:5])

[['Chittorgarh_Rajasthan', 'Ratnagiri_Maharashtra'], ['Pindwara_Rajasthan', 'Raipur_Chhattisgarh', 'Gokak_Karnataka'], ['Lucknow_Uttar Pradesh'], ['Lucknow_Uttar Pradesh'], ['Delhi_Delhi']]
